<a href="https://colab.research.google.com/github/Durga22-amie/-vqe-cancer-segmentation-/blob/main/GROUNDSTATE_ENERGY_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install "qiskit==1.2.4"
!pip install "qiskit-aer==0.14.2"
!pip install "qiskit-algorithms==0.3.0"


In [14]:
!pip install qiskit-algorithms
!pip install pylatexenc

In [15]:
!pip install pylatexenc


In [6]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
uploaded = files.upload()

Mounted at /content/drive


Saving 8_yes.jpg to 8_yes (1).jpg


In [16]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from qiskit.visualization import circuit_drawer
from qiskit import QuantumCircuit


# Qiskit core
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector

# Aer backend
from qiskit_aer.primitives import Estimator

# VQE + optimizer
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA

# Hamiltonian operator
from qiskit.quantum_info import SparsePauliOp


# --- 1. Load and process the MRI image to get 8 pixels ---
def load_and_select_pixels(image_path, num_pixels):
    try:
        image = Image.open(image_path).convert('L')
        img_array = np.array(image)
    except FileNotFoundError:
        print(f"Error: The image file '{image_path}' was not found.")
        return None

    np.random.seed(42)
    flat_indices = np.arange(img_array.size)
    selected_indices = np.random.choice(flat_indices, size=num_pixels, replace=False)
    selected_pixels = img_array.flatten()[selected_indices]
    normalized_pixels = selected_pixels / 255.0
    return normalized_pixels


mri_pixels = load_and_select_pixels('/content/8_yes.jpg', 8)
if mri_pixels is None:
    raise SystemExit()


# --- 2. Construct Hamiltonian ---
n_qubits = 8
hamiltonian_terms = []

for i, pixel_value in enumerate(mri_pixels):
    z_op = ['I'] * n_qubits
    z_op[i] = 'Z'
    pauli_string = ''.join(z_op)
    hamiltonian_terms.append((pauli_string, pixel_value))

hamiltonian = SparsePauliOp.from_list(hamiltonian_terms)


# --- 3. Custom Ansatz ---
def create_custom_ansatz(num_qubits, num_layers):
    params = ParameterVector('theta', length=num_qubits * num_layers * 2)
    custom_ansatz = QuantumCircuit(num_qubits)
    param_index = 0

    for layer in range(num_layers):
        # RY + RZ rotations
        for qubit in range(num_qubits):
            custom_ansatz.ry(params[param_index], qubit)
            param_index += 1
            custom_ansatz.rz(params[param_index], qubit)
            param_index += 1

        # CNOT chain
        for i in range(num_qubits - 1):
            custom_ansatz.cx(i, i+1)

        custom_ansatz.barrier()

    return custom_ansatz


custom_ansatz = create_custom_ansatz(n_qubits, num_layers=2)


# --- SAVE TEXT VERSION TO GOOGLE DRIVE ---
text = custom_ansatz.draw(output='text').single_string()

with open("/content/drive/MyDrive/circuit.txt", "w") as f:
    f.write(text)


# --- SAVE COLORED CIRCUIT DIAGRAM TO GOOGLE DRIVE ---
fig = circuit_drawer(custom_ansatz, output='mpl')
fig.savefig("/content/drive/MyDrive/circuit_colored.png", dpi=300)
qc = QuantumCircuit(8)

# Example: add some gates (replace with your ansatz)
for i in range(8):
    qc.ry(0.5, i)
    qc.rz(0.3, i)

for i in range(7):
    qc.cx(i, i+1)

# Draw with matplotlib (this gives purple gates + blue CNOTs)
circuit_image = qc.draw(output="mpl")

circuit_image


# --- 4. VQE ---
estimator = Estimator()
optimizer = SPSA(maxiter=100)

print("Running VQE to find the ground state energy...")

vqe = VQE(estimator, custom_ansatz, optimizer)
vqe_result = vqe.compute_minimum_eigenvalue(hamiltonian)


# --- 5. Results ---
final_energy = vqe_result.eigenvalue.real
expected_energy = -np.sum(mri_pixels)

print("VQE optimization completed.")
print(f"Final VQE Energy: {final_energy:.6f}")
print(f"Expected Minimum Energy: {expected_energy:.6f}")
print(f"Evaluations: {vqe_result.cost_function_evals}")


Running VQE to find the ground state energy...
VQE optimization completed.
Final VQE Energy: -1.359697
Expected Minimum Energy: -2.643137
Evaluations: 200
